## Testing inference

In [ ]:
from datasets import data_merge, data_merge_R
from transformers import *
import os
import torch
import cv2
from tqdm import tqdm
import numpy as np
from torch.utils.data import DataLoader
from networks import get_model
from networks import SSAN_M, SSAN_R
from utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data_dir = "/mnt/8TB/ml_projects_yeldar/"
data_bank = data_merge_R.data_merge(data_dir)
protocol = "Patchnet"
img_size = 112
map_size = 32
batch_size = 256
num_workers = 8

In [ ]:
PATH = './results/PatchNet/model/SSAN_R_pPatchnet_best.pth'
model = SSAN_R().cuda()
model.load_state_dict(torch.load(PATH)['state_dict'])

<All keys matched successfully>

In [ ]:
test_data_dic = data_bank.get_datasets(type='test', protocol=protocol, img_size=img_size, transform=transformer_test_video())
test_loader = DataLoader(test_data_dic, batch_size=batch_size, shuffle=False, num_workers=num_workers)

HEY PATCHNET
Loading PATCHNET, number: 6648
Total number: 6648


In [ ]:
model.eval()
with torch.no_grad():
    scores = []
    for i, sample_batched in enumerate(tqdm(test_loader)):
        image_x, label = sample_batched["image_x"].cuda(), sample_batched["label"].cuda()

        cls_x1_x1, fea_x1_x1, fea_x1_x2, _ = model(image_x, image_x)
        score_norm = torch.softmax(cls_x1_x1, dim=1)[:, 1]

        for ii in range(image_x.shape[0]):
            scores.append("{} {}\n".format(score_norm[ii], label[ii][0]))
                    
    map_score_val_filename = os.path.join('./', "{}_score.txt".format('test'))
    print("score: write test scores to {}".format(map_score_val_filename))
    with open(map_score_val_filename, 'w') as file:
        file.writelines(scores)

    test_ACC, fpr, FRR, HTER, auc_test, test_err = performances_val(map_score_val_filename)


  0%|          | 0/26 [00:00<?, ?it/s]

100%|██████████| 26/26 [00:03<00:00,  7.00it/s]

score: write test scores to ./test_score.txt


In [ ]:
test_ACC, fpr, FRR, HTER, auc_test, test_err

(0.9828519855595668,
 0.018272425249169437,
 0.016899338721528268,
 0.017585881985348852,
 0.9975945049687425,
 0.018272425249169437)